In [ ]:
import pandas as pd
import numpy as np
import math
from statistics import mode
import time
import sys
import tensorflow as tf
from tensorflow import keras
import random
import matplotlib.pyplot as plt
from memory_profiler import profile
from sklearn.preprocessing import StandardScaler
from math import isqrt

In [ ]:
# Get the list of available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')

if not gpus:
    print("No GPUs found. TensorFlow is using CPU.")
else:
    for gpu in gpus:
        print("GPU device name:", gpu.name)

In [ ]:
# Load the saved model
model = tf.keras.models.load_model('Training_output/Trained_10e_103L_001DF_Model.h5')

In [ ]:
# Initialize inputs and hyper parameters
state_size = 5
action_size = 2  # Accept or Reject
learning_rate = 0.001
epsilon = 1
decoy_rate = 0.99
epsilon_min = 0.1
gamma = 0.001

In [ ]:
#Initialize dataframes for outputs
epoch_train_accuracy_Calculated = []
detection_rate = []
each_reward = []
epoch_reward = []

In [ ]:
# Define a custom callback to measure training time and memory usage
class TimeMemoryCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.data = {'Epoch': [], 'Training Time': [], 'Memory RSS': [], 'Memory VMS': []}
        self.start_time = 0

    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_epoch_begin(self, epoch, logs=None):
        # Memory profiling for the beginning of each epoch
        memory_info = self.get_memory_usage()
        self.data['Epoch'].append(epoch)
        self.data['Memory RSS'].append(memory_info['rss'])
        self.data['Memory VMS'].append(memory_info['vms'])

    def on_train_end(self, logs=None):
        end_time = time.time()
        elapsed_time = end_time - self.start_time
        self.data['Training Time'] = [elapsed_time] * len(self.data['Epoch'])

    @staticmethod
    @profile

    def get_memory_usage():
        # This method is decorated with @profile to enable memory profiling
        import psutil

        process = psutil.Process()
        memory_info = process.memory_info()

        return {'rss': memory_info.rss / (1024 ** 2), 'vms': memory_info.vms / (1024 ** 2)}


In [ ]:
#Function for standardizing data
def stdInput(data):
    std_columns = ['sTotalSpeed','sxacc', 'beaconRate', 'DisDiff', 'rAvgSpeed']

# Extract the columns to be standardized
    data_to_standardize = data[std_columns]

# Initialize the StandardScaler
    scaler = StandardScaler()

# Fit the scaler to the data and transform the data
    scaled_data = scaler.fit_transform(data_to_standardize)

# Create a new DataFrame with the standardized data
    df_standardized = pd.DataFrame(scaled_data, columns=[f'{col}_std' for col in std_columns])

# Concatenate the new standardized columns with the original DataFrame
    data = pd.concat([data, df_standardized], axis=1)

    return data

In [ ]:
state_id = 0
#count for epsilon decays
ecount = 0
nid = state_id

cum_results = pd.DataFrame(columns = ['Attack','TP', 'TN', 'FP', 'FN', 'Accuracy', 'Precision', 'Recall', 'Fscore', 'FPR', 'MDR'])
all_each_results = pd.DataFrame(columns = ['Attack','TP', 'TN', 'FP', 'FN', 'Accuracy', 'Precision', 'Recall', 'Fscore', 'FPR', 'MDR'])


tp, fp, tn, fn = 0, 0, 0, 0
acc1, pre1, recall1, fs1, fpr1, mdr1 = 0, 0, 0, 0, 0, 0

with tf.device("/GPU:0"):
  for filetype in ['ConstPos', 'ConstPosOffset', 'RandomPos', 'RandomPosOffset', 'ConstSpeed', 'ConstSpeedOffset', 'RandomSpeed', 'RandomSpeedOffset', 'EventualStop', 'DelayedMessages', 'DoS', 'DoSRandom', 'DoSDisruptive', 'DataReplay', 'Disruptive', 'DataReplaySybil', 'GridSybil', 'DoSRandomSybil', 'DoSDisruptiveSybil']:

      #Read attack dataset in above sequence for testing
      sdata = pd.read_csv(f'DQN_Dataset_Sample/DQN_Testing_Data_Sample/Pair_CombinedSec_{filetype}.csv')
      sdata['DisDiff'] = abs(abs(sdata['SADis'])-abs(sdata['disCover']))
      sdata['ASLB'] = sdata['TSmean']- sdata['TSstd']
      sdata['ASUB'] = sdata['TSmean']+ sdata['TSstd']
      

      #Standardize the input data
      sdata = stdInput(sdata)
      
      test_tp, test_fp, test_tn, test_fn = 0, 0, 0, 0

      acc, pre, recall, fs, fpr, mdr = 0, 0, 0, 0, 0, 0


      #first find results in terms of accuracy, precision, recall, f1score and FPR
      each_results = pd.DataFrame(columns = ['TP', 'TN', 'FP', 'FN', 'Accuracy', 'Precision', 'Recall', 'Fscore', 'FPR', 'MDR'])
      
      urcvr = sdata.receiver.unique()
      Speed_buffer = pd.DataFrame(columns=['Receiver','Speed', 'Mean', 'STD', 'Exploit/Update'])
      all_overhead = pd.DataFrame(columns = ['RCount', 'MCount', 'Receiver', 'Memory_Usage', 'Time_Overhead'])
      rcount = 0
      for ri in urcvr:
        rdata = sdata[sdata.receiver==ri].sort_values(by='newTime').reset_index(drop=True)
        rcount+=1
        
        experience_replay = []
        for si in range(len(rdata)):
        # Replace with actual state data
          state = [rdata.loc[si,'beaconRate_std'],rdata.loc[si,'sTotalSpeed_std'], abs(rdata.loc[si,'sxacc_std']), rdata.loc[si,'DisDiff_std'], rdata.loc[si,'rAvgSpeed_std']]

          if si==0:
            Speed_buffer.loc[len(Speed_buffer)] = [rdata.loc[si,'receiver'], rdata.loc[si,'rAvgSpeed'],rdata.loc[si,'rAvgSpeed'], rdata.loc[si,'rAvgSpeed'], 'Exploit']
          else:
            # Calculate mean and std
            if si<5:
              wind = si
            else:
              wind = 10
            mean_value = rdata.loc[0:si,'rAvgSpeed'].shift().rolling(window=wind).mean().iloc[-1]
            std_value = rdata.loc[0:si,'rAvgSpeed'].shift().rolling(window=wind).std().iloc[-1]
            
            Speed_buffer.loc[len(Speed_buffer)] = [rdata.loc[si,'receiver'], rdata.loc[si,'rAvgSpeed'], mean_value, std_value+1, 'Exploit']

          
        # Check whether to test or update the model

          if Speed_buffer.loc[len(Speed_buffer)-1,'Mean']- Speed_buffer.loc[len(Speed_buffer)-1,'STD'] <= Speed_buffer.loc[len(Speed_buffer)-1,'Speed'] <= Speed_buffer.loc[len(Speed_buffer)-1,'Mean']+ Speed_buffer.loc[len(Speed_buffer)-1,'STD']:
            q_values = model.predict(np.array([state]))
            action = np.argmax(q_values)        

          else:
            if si<5:
              q_values = model.predict(np.array([state]))
              action = np.argmax(q_values)

            else:
              Speed_buffer.loc[len(Speed_buffer)-1, 'Exploit/Update'] = 'Update'
              batch_size = len(experience_replay) if len(experience_replay)<32 else 32

              if batch_size>1:
                minibatch = experience_replay[-batch_size:]
                states, actions, rewards = zip(*minibatch)

                # Convert the tuples to numpy arrays
                states = np.array(states)
                actions = np.array(actions)
                rewards = np.array(rewards)

                q_values = model.predict(states)

                # Calculate the target Q-values based on the current Q-values and rewards
                target_q_values = q_values.copy()

                for k in range(batch_size):
                  target_q_values[k][int(actions[k])] = rewards[k] + gamma * np.max(q_values[k])

                # Create the custom callback
                custom_callback = TimeMemoryCallback()
                # Update the Q-values using the DQN loss function
                
                # Example modification of the output layer
                # Modify the output layer for the new action size
                model.layers[-1] = keras.layers.Dense(action_size, activation='linear')

                # Optionally, freeze some or all pre-trained layers
                for layer in model.layers[:-1]:
                    layer.trainable = False

                # Compile the model for the new task
                model.compile(optimizer=keras.optimizers.Adam(0.2), loss='mse', metrics=['accuracy'])

                model.fit(states, target_q_values, epochs=2, verbose=0, callbacks=[custom_callback])


              q_values = model.predict(np.array([state]))
              action = np.argmax(q_values) 

              # Convert the collected data to a DataFrame
              df_metrics = pd.DataFrame(custom_callback.data)
              all_overhead.loc[len(all_overhead)] = [rcount, ri, si, df_metrics['Memory RSS'], df_metrics['Training Time']]

        actual_action = rdata.loc[si,'AttackerType']
            
        if (rdata.loc[si,'ASLB'] <= rdata.loc[si,'sTotalSpeed'] < rdata.loc[si,'ASUB']) and rdata.loc[si,'DisDiff']==0:
          rflag = 1
        else:
          rflag = 0

        rlag=1 if actual_action == 0 else 0
        if rflag == 1:
          reward = 1 if action == 0 else -1
        else:
          reward = -1 if action ==  0 else 1 

        if actual_action == action:
          if action == 0:
            test_tn+=1
          else:
            test_tp+=1
        else:
          if action == 0:
            test_fn+=1
          else:
            test_fp+=1

        each_reward.append(reward)
    #         experience_replay.append((state, action, reward, target_q_value))
        experience_replay.append((state, action, reward))
    
        if (test_tp+test_fp)!=0:
          pre = test_tp/(test_tp+test_fp)
        if (test_tp+test_fn)!=0:
          recall = test_tp/(test_tp+test_fn)
                  # tpr = tp/(tp+fn)
        if (test_tn+test_fp)!=0:
          fpr = test_fp/(test_tn+test_fp)
        if (test_tp+test_fn)!=0:
          mdr = test_fn/(test_tp+test_fn)
        acc = (test_tp+test_tn)/(test_tp+test_fp+test_tn+test_fn)
        if pre:
          if recall:
            fs = 2/((1/pre)+(1/recall))
        each_results.loc[len(each_results.index)] = [test_tp, test_tn, test_fp, test_fn, acc, pre, recall, fs, fpr, mdr]
      
      each_results.to_csv(f'Testing_output/Testing_103L_001DF_Results_{filetype}.csv') 
      Speed_buffer.to_csv(f'Testing_output/Testing_103L_001DF_SpeedBuffer_{filetype}.csv')
      all_overhead.to_csv(f'Testing_output/Testing_103L_001DF_Overhead_{filetype}.csv')

      all_each_results.loc[len(all_each_results.index)] = [filetype, test_tp, test_tn, test_fp, test_fn, acc, pre, recall, fs, fpr, mdr]
      tp+=test_tp
      tn+=test_tn
      fp+=test_fp
      fn+=test_fn     
        
      acc1 = (tp+tn)/(tp+tn+fp+fn)
      if (tp+fp)!=0:
        pre1 = tp/(tp+fp)
      if (tp+fn)!=0:
        recall1 = tp/(tp+fn)
      
      if (tn+fp)!=0:
        fpr1 = fp/(tn+fp)
      else:
        fpr1 = 0
      if (tp+fn)!=0:
        mdr1 = fn/(tp+fn)

      if pre1:
        if recall1:
          fs1 = 2/((1/pre1)+(1/recall1))

      cum_results.loc[len(cum_results.index)] = [filetype, tp, tn, fp, fn, acc1, pre1, recall1, fs1, fpr1, mdr1]
      model.save(f"Testing_output/Testing_103L_001DF_{filetype}_Model.h5")  # Save the model to a file

cum_results.to_csv('Testing_output/Testing_103L_001DF_CumulativeResults.csv')
all_each_results.to_csv('Testing_output/Testing_103L_001DF_EveryResults.csv')

  
